In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# Any results you write to the current directory are saved as output.

/kaggle/input/ifood-2019-fgvc6/sample_submission.csv
/kaggle/input/ifood-2019-fgvc6/class_list.txt
/kaggle/input/ifood-2019-fgvc6/train_set.zip
/kaggle/input/ifood-2019-fgvc6/train_labels.csv
/kaggle/input/ifood-2019-fgvc6/ifood2019_sample_submission.csv
/kaggle/input/ifood-2019-fgvc6/val_set.zip
/kaggle/input/ifood-2019-fgvc6/test_set.zip
/kaggle/input/ifood-2019-fgvc6/val_labels.csv


In [2]:
from zipfile import ZipFile
import shutil
import os

if not os.path.exists("./train_set"):
    for file_name in ['train_set.zip', 'val_set.zip', 'test_set.zip']:
        with ZipFile('../input/ifood-2019-fgvc6/' + file_name, 'r') as zipObj:
            print("unzipping", file_name)
            zipObj.extractall('./')

for dirname, _, _ in os.walk('./'):
    print(dirname)

unzipping train_set.zip
unzipping val_set.zip
unzipping test_set.zip
./
./val_set
./test_set
./train_set


In [3]:
df_train = pd.read_csv('../input/ifood-2019-fgvc6/train_labels.csv')
df_val   = pd.read_csv('../input/ifood-2019-fgvc6/val_labels.csv')
df_test = pd.read_csv('../input/ifood-2019-fgvc6/sample_submission.csv')

df_train['label'] = df_train['label'].astype(str)
df_val['label'] = df_val['label'].astype(str)

train_size = df_train.shape[0]
val_size = df_val.shape[0]
test_size = df_test.shape[0]
num_classes = df_train['label'].nunique()

In [4]:
for ind, item in df_train.iterrows():
    if not os.path.exists("./train_set/" + item['label']):
        os.mkdir("./train_set/" + item['label'])
    os.rename("./train_set/"+item['img_name'], "./train_set/"+item['label']+"/"+item['img_name'])
    
for ind, item in df_val.iterrows():
    if not os.path.exists("./val_set/" + item['label']):
        os.mkdir("./val_set/" + item['label'])
    os.rename("./val_set/"+item['img_name'], "./val_set/"+item['label']+"/"+item['img_name'])

os.mkdir("./test_set/0")
for ind, item in df_test.iterrows():
    os.rename("./test_set/"+item['img_name'], "./test_set/0/"+item['img_name'])

In [5]:
import torch
import torchvision
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler

NUM_TRAIN = 10000
NUM_VAL = 1000
BATCH_SIZE = 64

transform_train = T.Compose([
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_train = torchvision.datasets.ImageFolder(
                root="./train_set",
                transform=transform_train
            )
# loader_train = DataLoader(data_train, batch_size=BATCH_SIZE, 
#                           sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
loader_train = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=True)
data_val = torchvision.datasets.ImageFolder(
                root="./val_set",
                transform=transform_test
            )
# loader_val = DataLoader(data_val, batch_size=BATCH_SIZE,
#                        sampler=sampler.SubsetRandomSampler(range(NUM_VAL)))
loader_val = DataLoader(data_val, batch_size=BATCH_SIZE, shuffle=False)
data_test = torchvision.datasets.ImageFolder(
                root="./test_set",
                transform=transform_test
            )
loader_test = DataLoader(data_test, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
images, labels = next(iter(loader_train))
print("batch data shape: ", images.shape)
print("batch label shape: ", labels.shape)

batch data shape:  torch.Size([64, 3, 224, 224])
batch label shape:  torch.Size([64])


In [7]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [8]:
import time

def train(model, optimizer, epochs=1, print_every=100):
    global best_loss
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    model.train() 

    since = time.time()
    for e in range(epochs):
        print('Epoch %d, time_elapsed = %d' % (e, time.time()-since))
        train_loss = 0
        for t, (img, _) in enumerate(loader_train):
            img = img.to(device=device, dtype=dtype)

            # forward
            output = model(img)
            loss = criterion(output, img)
            train_loss += loss.item()*img.size(0)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print('train: loss (%.2f)' % (train_loss/train_size))
        val_loss = evaluate(loader_val, model)
        print()
        
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), 'autoencoder.pth.tar')

In [9]:
def evaluate(loader, model):
    model.eval()  # set model to evaluation mode
    
    with torch.no_grad():
        for img, _ in loader:
            img = img.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            output = model(img)
            loss = criterion(output, img)

        print('val: loss (%.2f)' % loss)
        return loss

In [10]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, (3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 16, (3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 8, (3, 3), padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # 8*28*28
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 8, (3, 3), stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 16, (3, 3), stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 32, (3, 3), stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, (3, 3), padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


learning_rate = 1e-3
model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
best_loss = 1000
train(model, optimizer, epochs=10, print_every=100)

Epoch 0, time_elapsed = 0
train: loss (0.44)
val: loss (0.32)

Epoch 1, time_elapsed = 871
train: loss (0.37)
val: loss (0.31)

Epoch 2, time_elapsed = 1739
train: loss (0.36)
val: loss (0.31)

Epoch 3, time_elapsed = 2615
train: loss (0.36)
val: loss (0.30)

Epoch 4, time_elapsed = 3495
train: loss (0.36)
val: loss (0.30)

Epoch 5, time_elapsed = 4378
train: loss (0.36)
val: loss (0.30)

Epoch 6, time_elapsed = 5266
train: loss (0.36)
val: loss (0.30)

Epoch 7, time_elapsed = 6141
train: loss (0.35)
val: loss (0.30)

Epoch 8, time_elapsed = 7018
train: loss (0.35)
val: loss (0.30)

Epoch 9, time_elapsed = 7898
train: loss (0.35)
val: loss (0.30)



In [12]:
# y_pred = predict(loader_test, model)
# df_test['label'] = [' '.join(item) for item in y_pred.cpu().numpy().astype(str)]
# df_test.to_csv("submission.csv", index=False)

In [13]:
for path in ["./train_set", "./val_set", "./test_set"]:
    shutil.rmtree(path)